In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Import

In [ ]:
from scipy.sparse import hstack
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression,SGDRegressor, Ridge
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, cross_val_score
import scipy
import time
import re
import pandas as pd
from nltk.corpus import stopwords
import spacy
from sklearn import preprocessing as preprocessingsk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import text
from Orange.data import Table
from Orange.preprocess import Impute, Model
from Orange.modelling import TreeLearner
from Orange.data.pandas_compat import table_from_frame,table_to_frame
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.impute import KNNImputer

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
#import category_encoders as cat

import nltk
nltk.download('wordnet')
nltk.download('stopwods')
nltk.download('punkt')

In [ ]:
!pip install orange3



## Methods and utils

In [ ]:
##################################  
## General methods
##################################

def loadData(directory):
    df = pd.read_csv(directory,sep="\t")
    return df

def get_final_csv(ids, y, filename):
    pd.DataFrame(dict(Id = ids,Predicted = y)).to_csv(filename,sep=",",index=False)

    
models = [LinearRegression(),Ridge(), SGDRegressor()]
targets = ["LinearRegression","Ridge","SGDRegressor"]
def evaluateModels(models, targets,X,y):
    scores = pd.DataFrame()
    for model,target in zip(models,targets):
        scores[target] = cross_val_score(model, X, y, scoring='r2', cv=3, n_jobs=-1)
        
    return scores



##################################  
## Encoding and Missing values
##################################
def impute_gender(X_dev,X_eval):
    X_dev['mean_of_four_gender'] = (X_dev['aroma'] + X_dev['appearance'] + X_dev['palate'] + X_dev['taste'] )/4
    x_m = X_dev[X_dev['gender'] == "Male"]
    x_f = X_dev[X_dev['gender'] == "Female"]
    meanm = (x_m['aroma'].mean() + x_m['appearance'].mean() + x_m['palate'].mean() + x_m['taste'].mean() )/4
    meanf = (x_f['aroma'].mean() + x_f['appearance'].mean() + x_f['palate'].mean() + x_f['taste'].mean() )/4
    X_dev['gender'].loc[(abs(X_dev['mean_of_four_gender'] - meanm) 
                         <= abs(X_dev['mean_of_four_gender'] - meanf)  ) & (X_dev['gender'].isnull())] = 10  
    X_dev['gender'].loc[(abs(X_dev['mean_of_four_gender'] - meanm) 
                         > abs(X_dev['mean_of_four_gender'] - meanf)  ) & (X_dev['gender'].isnull())] = 20  
    X_dev['gender'].loc[(X_dev['gender'] ==  10)] = "Male"
    X_dev['gender'].loc[(X_dev['gender'] ==  20)] = "Female"
    X_dev['gender'].loc[(X_dev['gender'] ==  'Male')] = 0
    X_dev['gender'].loc[(X_dev['gender'] ==  'Female')] = 1
    ################################################
    ################################################
    X_eval['mean_of_four_gender'] = (X_eval['aroma'] + X_eval['appearance'] + X_eval['palate'] + X_eval['taste'] )/4
    x_m = X_eval[X_eval['gender'] == "Male"]
    x_f = X_eval[X_eval['gender'] == "Female"]
    meanm = (x_m['aroma'].mean() + x_m['appearance'].mean() + x_m['palate'].mean() + x_m['taste'].mean() )/4
    meanf = (x_f['aroma'].mean() + x_f['appearance'].mean() + x_f['palate'].mean() + x_f['taste'].mean() )/4
    X_eval['gender'].loc[(abs(X_eval['mean_of_four_gender'] - meanm) 
                         <= abs(X_eval['mean_of_four_gender'] - meanf)  ) & (X_eval['gender'].isnull())] = 10  
    X_eval['gender'].loc[(abs(X_eval['mean_of_four_gender'] - meanm) 
                         > abs(X_eval['mean_of_four_gender'] - meanf)  ) & (X_eval['gender'].isnull())] = 20  
    X_eval['gender'].loc[(X_eval['gender'] ==  10)] = "Male"
    X_eval['gender'].loc[(X_eval['gender'] ==  20)] = "Female"
    X_eval['gender'].loc[(X_eval['gender'] ==  'Male')] = 0
    X_eval['gender'].loc[(X_eval['gender'] ==  'Female')] = 1
    return X_dev['gender'],X_eval['gender']

    ##########################
    ##########################
def impute_orange(X_dev,X_eval):
    X_dev['gender'].loc[(X_dev['gender'] ==  'Male')] = 0
    X_dev['gender'].loc[(X_dev['gender'] ==  'Female')] = 0
    
    data = Table(X_dev.drop(columns=["name","style","birthdayRaw","profileName"]))
    imputer = Impute(method=Model(TreeLearner()))
    impute_heart = imputer(data)
    df= table_to_frame(impute_heart)
    X_dev["ABV"] = df["Feature 1"]
    X_dev["appearance"] = df["Feature 2"]
    X_dev["aroma"] = df["Feature 3"]
    X_dev["overall"] = df["Feature 4"]
    X_dev["palate"] = df["Feature 5"]
    X_dev["taste"] = df["Feature 6"]
    X_dev["ageInSeconds"] = df["Feature 7"]
    X_dev["birthdayUnix"] = df["Feature 8"]
    X_dev["gender"] = df["Feature 9"]
    ##################
    ##################
    X_eval['gender'].loc[(X_eval['gender'] ==  'Male')] = 0
    X_eval['gender'].loc[(X_eval['gender'] ==  'Female')] = 0
    
    data = Table(X_eval.drop(columns=["name","style","birthdayRaw","profileName"]))
    imputer = Impute(method=Model(TreeLearner()))
    impute_heart = imputer(data)
    df= table_to_frame(impute_heart)
    X_eval["ABV"] = df["Feature 1"]
    X_eval["appearance"] = df["Feature 2"]
    X_eval["aroma"] = df["Feature 3"]
    X_eval["palate"] = df["Feature 4"]
    X_eval["taste"] = df["Feature 5"]
    X_eval["ageInSeconds"] = df["Feature 6"]
    X_eval["birthdayUnix"] = df["Feature 7"]
    X_eval["gender"] = df["Feature 8"]
    return X_dev,X_eval
    ##########################
    ##########################
def MonthTonum(str):
    if(str == 'Jan'):return 1
    if(str == 'Feb'):return 2
    if(str == 'Mar'):return 3
    if(str == 'Apr'):return 4
    if(str == 'May'):return 5
    if(str == 'Jun'):return 6
    if(str == 'Jul'):return 7
    if(str == 'aug'):return 8
    if(str == 'Sep'):return 9
    if(str == 'Oct'):return 10
    if(str == 'Nov'):return 11
    if(str == 'Dec'):return 12
    else: return 0
###########################
###########################
###########################
def KNNimp(X_dev,X_eval):
    print("X_dev KNNIMPUR STARTED")
    X_dev['birthdayRaw'].loc[(X_dev['birthdayRaw'].isna()==True)] = '0'
    X_dev['birthdayRaw'] = X_dev['birthdayRaw'].apply(lambda int_date: (int(((str(int_date)).split(',')[0])[-2:].strip() + (str(int_date))[-4:]+str((MonthTonum(((str(int_date)).split(',')[0].split(' ')[0].strip())))))))
    X_dev['birthdayRaw'].loc[(X_dev['birthdayRaw']==0)] = np.nan
    
    
    X_dev['gender'].loc[(X_dev['gender'] ==  'Male')] = 0
    X_dev['gender'].loc[(X_dev['gender'] ==  'Female')] = 1
    
    df_dev = X_dev
    df_dev = df_dev.drop(columns = ['text','name','profileName','style'])
    
   
    
    scaler = MinMaxScaler()
    df1 = df_dev
    df1 = df1.drop(columns = ['overall'])
    df1 = pd.DataFrame(scaler.fit_transform(df1), columns = df1.columns)
    df_dev = pd.concat([df_dev['overall'], df1], axis=1)
    

    imputer = KNNImputer(n_neighbors=500000)
    df_dev = pd.DataFrame(imputer.fit_transform(df_dev),columns = df_dev.columns)
    print("X_dev KNNIMPUR ENDED")
    ###################
    print("X_eval KNNIMPUR STARTED")

    X_eval['birthdayRaw'].loc[(X_eval['birthdayRaw'].isna()==True)] = '0'
    X_eval['birthdayRaw'] = X_eval['birthdayRaw'].apply(lambda int_date: (int(((str(int_date)).split(',')[0])[-2:].strip() + (str(int_date))[-4:]+str((MonthTonum(((str(int_date)).split(',')[0].split(' ')[0].strip())))))))
    X_eval['birthdayRaw'].loc[(X_eval['birthdayRaw']==0)] = np.nan
    
    X_eval['gender'].loc[(X_eval['gender'] ==  'Male')] = 0
    X_eval['gender'].loc[(X_eval['gender'] ==  'Female')] = 1
    
    df_eval = X_eval
    df_eval = df_eval.drop(columns = ['text','name','profileName','style'])

    
    
    
    scaler = MinMaxScaler()
    df_eval = pd.DataFrame(scaler.fit_transform(df_eval), columns = df_eval.columns)
    df_eval.head()

    imputer = KNNImputer(n_neighbors=50)
    df_eval = pd.DataFrame(imputer.fit_transform(df_eval),columns = df_eval.columns)
    print("X_eval KNNIMPUR ENDED")
    return df_dev,df_eval
    
def preprocessing(X_d,X_e):
    # region_2 has too many duplicates
    # description will be manipulated separately
    imputer2 = SimpleImputer(missing_values=np.nan, strategy='mean')
    #X_d["overall"] = imputer2.fit_transform(np.array(X_d["overall"]).reshape(-1,1))
    
    X_d1,X_e1 = KNNimp(X_d,X_e)

    
    imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imputer2 = SimpleImputer(missing_values=np.nan, strategy='mean')
    #I didn't use Simple imputer or Orange and isntead I used KNN
    #X_d["birthdayRaw"] = imputer.fit_transform(np.array(X_d["birthdayRaw"]).reshape(-1,1))
    #X_d["gender"] = imputer.fit_transform(np.array(X_d["gender"]).reshape(-1,1))
    #X_d["profileName"] = imputer.fit_transform(np.array(X_d["profileName"]).reshape(-1,1))
    #X_d["ABV"] = imputer.fit_transform(np.array(X_d["ABV"]).reshape(-1,1))
    #X_d["ageInSeconds"]  = imputer2.fit_transform(np.array(X_d["ageInSeconds"] / 60 / 60 / 24 / 365.25).reshape(-1,1))
    #X_d["ageInSeconds"] = round(X_d["ageInSeconds"], 5)
    #X_d["birthdayUnix"]  = imputer2.fit_transform((np.array(X_d["birthdayUnix"])).reshape(-1,1))
    #X_d["birthdayUnix"] = preprocessingsk.normalize((np.array(X_d["birthdayUnix"])).reshape(-1,1),'max',0)
    #X_d["birthdayUnix"] = round(X_d["birthdayUnix"], 5)
    #################################
    #X_e["birthdayRaw"] = imputer.fit_transform(np.array(X_e["birthdayRaw"]).reshape(-1,1))
    #X_e["gender"] = imputer.fit_transform(np.array(X_e["gender"]).reshape(-1,1))
    #X_e["profileName"] = imputer.fit_transform(np.array(X_e["profileName"]).reshape(-1,1))
    #X_e["ABV"] = imputer.fit_transform(np.array(X_e["ABV"]).reshape(-1,1))
    #X_e["ageInSeconds"]  = imputer2.fit_transform(np.array(X_e["ageInSeconds"] / 60 / 60 / 24 / 365.25).reshape(-1,1))
    #X_e["ageInSeconds"] = round(X_e["ageInSeconds"], 5)
    #X_e["birthdayUnix"]  = imputer2.fit_transform((np.array(X_e["birthdayUnix"])).reshape(-1,1))
    #X_e["birthdayUnix"] = preprocessingsk.normalize((np.array(X_e["birthdayUnix"])).reshape(-1,1),'max',0)
    #X_e["birthdayUnix"] = round(X_e["birthdayUnix"], 5)



    X_d = pd.concat([X_d1, X_d[['name','profileName','style']]], axis=1)
    X_e = pd.concat([X_e1, X_e[['name','profileName','style']]], axis=1)
    y = X_d.overall
    X_d = X_d.drop(columns=["overall"])
   
    # concat dev and eval for the encoding 


    df = pd.concat([X_d,X_e])
    
    # encode and use the sparse matrix because pandas' df is too heavy
    df_enc = pd.get_dummies(df)
    
    df_enc_scipy = scipy.sparse.csr_matrix(df_enc.values)
    
    # split and return the encoded values
    return df_enc_scipy[:X_d.shape[0]], y, df_enc_scipy[X_d.shape[0]:],df_enc

In [ ]:
##################################  
## Document preprocessing
##################################

from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as sw

# general structure learnt from Lab10
class LemmaTokenizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        
    def __call__(self, document):
        lemmas = []
        
        for t in word_tokenize(document):
            
            # remove external spaces
            t = t.strip()
            # lowercase
            t = t.lower()
            # remove punctuation
            t = re.sub(r'[^\w\s]','',t)
            # remove numbers
            t = re.sub(r'[\d+]','',t)
            
            lemma = self.lemmatizer.lemmatize(t)
            if len(lemma) > 1:
                lemmas.append(lemma)
    
        return lemmas

def preprocessText(text_train,text_test):
    vectorizer = TfidfVectorizer(ngram_range=(2,4), tokenizer=LemmaTokenizer(), stop_words="english")
    
    # fit the TfidfVectorizer on the development set and transform it on both dev and eval
    
    dev_vec = vectorizer.fit_transform(text_train.text.values.astype('U'))
    eval_vec = vectorizer.transform(text_test.text.values.astype('U'))

    return dev_vec, eval_vec

## Load data

In [ ]:
# load datasets

X_dev = loadData('DSL/development.tsv')
X_eval = loadData('DSL/evaluation.tsv')

In [ ]:
X_dev = X_dev.rename(columns={'beer/ABV':'ABV','beer/name':'name','beer/style':'style','review/appearance':'appearance',
                      'review/aroma':'aroma','review/palate':'palate','review/taste':'taste','review/text':'text',
                      'user/ageInSeconds':'ageInSeconds','user/birthdayRaw':'birthdayRaw','user/gender':'gender',
                      'user/profileName':'profileName','review/overall':'overall','user/birthdayUnix':'birthdayUnix'
                     })
X_eval = X_eval.rename(columns={'beer/ABV':'ABV','beer/name':'name','beer/style':'style','review/appearance':'appearance',
                      'review/aroma':'aroma','review/palate':'palate','review/taste':'taste','review/text':'text',
                      'user/ageInSeconds':'ageInSeconds','user/birthdayRaw':'birthdayRaw','user/gender':'gender',
                      'user/profileName':'profileName','review/overall':'overall','user/birthdayUnix':'birthdayUnix'
                     })



## Preprocessing β

In [ ]:
%%time

####################################################
## Preprocessing - without removing the outliers : β
####################################################
#X_dev[["text"]].type()
# drop duplicates
#X_dev = X_dev.drop_duplicates()
#X_dev = X_dev[X_dev.isnull().sum(axis=1)>=4 ]
#X_eval = X_eval[X_eval.isnull().sum(axis=1)>=4 ]
# encode the categorical features and handle np.nan
X_dev_prep, y, X_eval_prep,xx = preprocessing(X_dev,X_eval)

#preprocess the descriptions
dev_vec, eval_vec = preprocessText(X_dev[["text"]].copy(),
                                   X_eval[["text"]].copy())

# concat the encoded df and the tf-idf
X_conc_dev = hstack((X_dev_prep, dev_vec))
X_conc_eval = hstack((X_eval_prep, eval_vec))

## Model evaluation

In [ ]:
## for the instructor : you can skip this script

####################################################
## Evaluate β
####################################################

scores = evaluateModels(models,targets,X_conc_dev,y)

np.mean(scores)

## Hyperparameters tuning

In [ ]:
def doGridSearch(model,hyperparams,X,y):
    
    gs = GridSearchCV(estimator=model,param_grid=hyperparams,
                      scoring='r2',cv=3, n_jobs=4,verbose=True)
    gs.fit(X, y)
    
    return gs

In [ ]:

####################################################
## Grid search linear regression
####################################################

hyperparams_LR = {
    'fit_intercept' : [True,False],
    'normalize' : [True,False]
}

gs_lr = doGridSearch(LinearRegression(),hyperparams_LR,X_conc_dev,y)

print(f"Best params:\t{gs_lr.best_params_}")
print(f"Best score:\t{gs_lr.best_score_}")

y_pred_lr = gs_lr.predict(X_conc_eval)
get_final_csv(list(X_eval.index),y_pred_lr,"submit-linear-regression.csv")

## 0.698 
## {'fit_intercept': True, 'normalize': False}

In [ ]:
%time

####################################################
## Grid search Ridge
####################################################

hyperparams_Ridge = {
    'alpha' : [0.01,0.1,1,8,10,12,14,16,18,19,20,21,23,25,27,30,35]
    #'alpha' : [0.01,0.1,19]

    
}

gs_ridge = doGridSearch(Ridge(),hyperparams_Ridge,X_conc_dev,y)
print(f"Best params:\t{gs_ridge.best_params_}")
print(f"Best score:\t{gs_ridge.best_score_}")

y_pred_sgd = gs_ridge.predict(X_conc_eval)
get_final_csv(list(X_eval.index),y_pred_sgd,"submit-ridge3.csv")

## 0.711
## {'alpha': 0.01}

In [ ]:

####################################################
## Grid search SGD Regressor
####################################################

hyperparams_SGD = {
    'loss' : ['squared_loss'],
    'penalty' : ['l1','l2'],
    'alpha' : np.logspace(-5, 0, 6),
    'eta0' : [0.01, 0.1]
}

gs_sgd = doGridSearch(SGDRegressor(max_iter=10000),hyperparams_SGD,X_conc_dev,y)
print(f"Best params:\t{gs_sgd.best_params_}")
print(f"Best score:\t{gs_sgd.best_score_}")


## 0.682
## {'alpha': 1e-05, 'eta0': 0.1, 'loss': 'squared_loss', 'penalty': 'l1'}

In [ ]:
hyperparams_RF = {
"n_estimators": [1,2],
"criterion": ["mse", "mae"],
"max_features": ["auto"],
"random_state": [42], # always use the samet random seed
"n_jobs": [-1], # for parallelization
}
gs_RF = doGridSearch(RandomForestRegressor(), hyperparams_RF,X_conc_dev,y)
print(f"Best params:\t{gs_RF.best_params_}")
print(f"Best score:\t{gs_RF.best_score_}")

In [ ]:

model = RandomForestRegressor(10, random_state=42)
model.fit(X_conc_dev,y)
y_pred = model.predict(X_conc_eval)
get_final_csv(list(X_eval.index),y_pred,"submit3.csv")


## Final prediction 

In [ ]:
%%time



model = Ridge(alpha = 10)
model.fit(X_conc_dev,y)
y_pred = model.predict(X_conc_eval)

get_final_csv(list(X_eval.index),y_pred,"submit.csv")

